In [81]:
# Usage guide:

# This document is not kept on record. If the calcualtions are to be kept,
# copy the whole notebook and put it in "quick_calculations" or appropriate
# folders.

In [82]:
# import all the E9 stuff
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import time

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_atom as E9a
import E9_fn.E9_cooltrap as E9ct
import E9_fn.plane_wave_expansion.blochstate_class as bsc
import E9_fn.polarizabilities_calculation as E9pol
# import E9_fn.datasets.transition_line_data as TLData
from E9_fn import util

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

# Conversion between driving strengths

In [83]:
for E_R, m_atom, label in zip(
    [E9c.E_R532_Rb87,   E9c.E_R1064_Rb87,   E9c.E_R532_K40, E9c.E_R1064_K40],
    [E9c.m_Rb87,        E9c.m_Rb87,         E9c.m_K40,      E9c.m_K40],
    ["532_Rb87",        "1064_Rb87",        "532_K40",      "1064_K40"]
):
    f_R_over_1kHz = E_R / E9c.hnobar / 1e3
    F_0a = np.pi * (E_R / E9c.hnobar) * E9c.a_sw_tri**2 * m_atom / E9c.hbar
    print(f"{label}:")
    print(f"    E_R / h     = {f_R_over_1kHz:.6f} kHz")
    print(f"    F_0a        = {F_0a:.6f} dimless")
    print(f"    F_0v        = {F_0a / f_R_over_1kHz:.6f} f dimless")
    print(f"    F_0r        = {F_0a / f_R_over_1kHz**2:.6f} f^2 dimless")
    print(f"    M_v / M_a   = {f_R_over_1kHz:.6f} f^-1 dimless")
    print(f"    M_r / M_a   = {f_R_over_1kHz**2:.6f} f^-2 dimless")

532_Rb87:
    E_R / h     = 8.111256 kHz
    F_0a        = 4.386491 dimless
    F_0v        = 0.540791 f dimless
    F_0r        = 0.066672 f^2 dimless
    M_v / M_a   = 8.111256 f^-1 dimless
    M_r / M_a   = 65.792473 f^-2 dimless
1064_Rb87:
    E_R / h     = 2.027814 kHz
    F_0a        = 1.096623 dimless
    F_0v        = 0.540791 f dimless
    F_0r        = 0.266686 f^2 dimless
    M_v / M_a   = 2.027814 f^-1 dimless
    M_r / M_a   = 4.112030 f^-2 dimless
532_K40:
    E_R / h     = 17.639438 kHz
    F_0a        = 4.386491 dimless
    F_0v        = 0.248675 f dimless
    F_0r        = 0.014098 f^2 dimless
    M_v / M_a   = 17.639438 f^-1 dimless
    M_r / M_a   = 311.149770 f^-2 dimless
1064_K40:
    E_R / h     = 4.409859 kHz
    F_0a        = 1.096623 dimless
    F_0v        = 0.248675 f dimless
    F_0r        = 0.056391 f^2 dimless
    M_v / M_a   = 4.409859 f^-1 dimless
    M_r / M_a   = 19.446861 f^-2 dimless


# Hole dynamics

In [94]:
q_width = 0.2                   # (quasi-)momentum width of the hole, in k_BZ
P_ODTa = 0.05                   # power of ODTa, 0.05 measured 20250903
P_ODTb = 0.23                   # power of ODTb, 0.23 measured 20250903
# For lattice beams, I take the combined trap depths to be the same as the lattice depths
V_1064 = 30.89e3 * E9c.hnobar   # lattice depth of 1064 beam
V_532 = 0e3 * E9c.hnobar       # lattice depth of 532 beam
# effective trap depth is halved if the lattice beams are far detuned (and thus not forming a lattice)
bool_1064_detuned = False
bool_532_detuned = True

## ODT potentials
To add the trap frequencies incoherently one use the rms.

In [95]:
print("Assume K40 atoms")
# The choice of axis convention doesn't affect f_trap, since it is the rms of three orthogonal trap frequencies

conventions = [
    {
        "axis_convention": "x_EW",
        "description": "x = EW, y = NS",
        "theta_x_ODTa": np.pi / 6,
        "theta_y_ODTa": np.pi / 3,
        "theta_x_ODTb": np.pi / 6,
        "theta_y_ODTb": np.pi / 3,
    },
    {
        "axis_convention": "x_ODTa_waist",
        "description": "x = ODTa waist, y = ODTa optical axis",
        "theta_x_ODTa": np.pi / 2,
        "theta_y_ODTa": 0.0,
        "theta_x_ODTb": np.pi / 6,
        "theta_y_ODTb": np.pi / 3,
    },
    {
        "axis_convention": "x_ODTb_waist",
        "description": "x = ODTb waist, y = ODTb optical axis",
        "theta_x_ODTa": np.pi / 6,
        "theta_y_ODTa": np.pi / 3,
        "theta_x_ODTb": np.pi / 2,
        "theta_y_ODTb": 0.0,
    },
]

lamb_ODT = 1064e-9
K_pol_1064 = E9pol.alpha_s_K_4S1o2(lamb_ODT)
I_max_ODTa = util.I_from_power(P_ODTa, E9c.w0_ODTa_hori, E9c.w0_ODTa_vert)
I_max_ODTb = util.I_from_power(P_ODTb, E9c.w0_ODTb_hori, E9c.w0_ODTb_vert)
V_max_ODTa = E9pol.I2J_from_pol(I_max_ODTa, K_pol_1064)
V_max_ODTb = E9pol.I2J_from_pol(I_max_ODTb, K_pol_1064)
print(f"V_max_ODTa = {V_max_ODTa / E9c.k_B * 1e6:.3f} uK = {V_max_ODTa / E9c.hnobar / 1e3:.3f} kHz")
print(f"V_max_ODTb = {V_max_ODTb / E9c.k_B * 1e6:.3f} uK = {V_max_ODTb / E9c.hnobar / 1e3:.3f} kHz\n")

rows = []
for cfg in conventions:
    # Unpack angles
    tx_a = cfg["theta_x_ODTa"]
    ty_a = cfg["theta_y_ODTa"]
    tx_b = cfg["theta_x_ODTb"]
    ty_b = cfg["theta_y_ODTb"]

    wx_ODTa = np.sqrt(-2 * V_max_ODTa * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTa_hori, E9c.w0_ODTa_vert, theta = tx_a) / E9c.m_K40)
    wy_ODTa = np.sqrt(-2 * V_max_ODTa * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTa_hori, E9c.w0_ODTa_vert, theta = ty_a) / E9c.m_K40)
    wz_ODTa = np.sqrt(-2 * V_max_ODTa * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTa_hori, E9c.w0_ODTa_vert, theta = np.pi / 2, waxis = "y") / E9c.m_K40)
    wx_ODTb = np.sqrt(-2 * V_max_ODTb * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTb_hori, E9c.w0_ODTb_vert, theta = tx_b) / E9c.m_K40)
    wy_ODTb = np.sqrt(-2 * V_max_ODTb * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTb_hori, E9c.w0_ODTb_vert, theta = ty_b) / E9c.m_K40)
    wz_ODTb = np.sqrt(-2 * V_max_ODTb * util.x2approx_gaussian_beam_3D(lamb_ODT, E9c.w0_ODTb_hori, E9c.w0_ODTb_vert, theta = np.pi / 2, waxis = "y") / E9c.m_K40)
    wx_all = np.sqrt(wx_ODTa**2 + wx_ODTb**2)
    wy_all = np.sqrt(wy_ODTa**2 + wy_ODTb**2)
    wz_all = np.sqrt(wz_ODTa**2 + wz_ODTb**2)
    wxy_all = np.sqrt(wx_all**2 + wy_all**2)
    w_all = np.sqrt(wxy_all**2 + wz_all**2)

    f_x_ODTa = wx_ODTa / 2 / np.pi; f_y_ODTa = wy_ODTa / 2 / np.pi; f_z_ODTa = wz_ODTa / 2 / np.pi
    f_x_ODTb = wx_ODTb / 2 / np.pi; f_y_ODTb = wy_ODTb / 2 / np.pi; f_z_ODTb = wz_ODTb / 2 / np.pi
    f_x_all = wx_all / 2 / np.pi; f_y_all = wy_all / 2 / np.pi; f_z_all = wz_all / 2 / np.pi
    f_xy_all = wxy_all / 2 / np.pi; f_all = w_all / 2 / np.pi

    rows.append({
        "axis_convention": cfg["axis_convention"],
        "description": cfg["description"],

        # angles (deg) for readability
        "theta_x_ODTa_deg": np.rad2deg(tx_a),
        "theta_y_ODTa_deg": np.rad2deg(ty_a),
        "theta_x_ODTb_deg": np.rad2deg(tx_b),
        "theta_y_ODTb_deg": np.rad2deg(ty_b),

        # angular freqs (rad/s)
        "wx_ODTa_rad_s": wx_ODTa, "wy_ODTa_rad_s": wy_ODTa, "wz_ODTa_rad_s": wz_ODTa,
        "wx_ODTb_rad_s": wx_ODTb, "wy_ODTb_rad_s": wy_ODTb, "wz_ODTb_rad_s": wz_ODTb,
        "wx_all_rad_s": wx_all,   "wy_all_rad_s": wy_all,   "wz_all_rad_s": wz_all,
        "wr_all_rad_s": wxy_all,   "w_all_rad_s": w_all,

        # frequencies (Hz)
        "f_x_ODTa_Hz": f_x_ODTa, "f_y_ODTa_Hz": f_y_ODTa, "f_z_ODTa_Hz": f_z_ODTa,
        "f_x_ODTb_Hz": f_x_ODTb, "f_y_ODTb_Hz": f_y_ODTb, "f_z_ODTb_Hz": f_z_ODTb,
        "f_x_all_Hz": f_x_all,   "f_y_all_Hz": f_y_all,   "f_z_all_Hz": f_z_all,
        "f_xy_all_Hz": f_xy_all,   "f_all_Hz": f_all,
    })
df = pd.DataFrame(rows)

rows_to_print = ["axis_convention",
                 "theta_x_ODTa_deg", "theta_y_ODTa_deg", "theta_x_ODTb_deg", "theta_y_ODTb_deg",
                 "f_x_ODTa_Hz", "f_y_ODTa_Hz", "f_z_ODTa_Hz",
                 "f_x_ODTb_Hz", "f_y_ODTb_Hz", "f_z_ODTb_Hz",
                 "f_x_all_Hz", "f_y_all_Hz", "f_z_all_Hz", "f_xy_all_Hz", "f_all_Hz",]
df_T = df.filter(items = rows_to_print).set_index('axis_convention').T
with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.width", 220):
    print(df_T.to_string(float_format=lambda x: f"{x:.4f}"))

Assume K40 atoms
V_max_ODTa = -1.536 uK = -32.003 kHz
V_max_ODTb = -5.388 uK = -112.270 kHz

axis_convention      x_EW  x_ODTa_waist  x_ODTb_waist
theta_x_ODTa_deg  30.0000       90.0000       30.0000
theta_y_ODTa_deg  60.0000        0.0000       60.0000
theta_x_ODTb_deg  30.0000       30.0000       90.0000
theta_y_ODTb_deg  60.0000       60.0000        0.0000
f_x_ODTa_Hz       27.1223       54.1907       27.1223
f_y_ODTa_Hz       46.9358        1.3958       46.9358
f_z_ODTa_Hz      216.3509      216.3509      216.3509
f_x_ODTb_Hz      104.4866      104.4866      208.9688
f_y_ODTb_Hz      180.9727      180.9727        0.7808
f_z_ODTb_Hz      150.1043      150.1043      150.1043
f_x_all_Hz       107.9494      117.7034      210.7216
f_y_all_Hz       186.9601      180.9781       46.9422
f_z_all_Hz       263.3230      263.3230      263.3230
f_xy_all_Hz      215.8869      215.8869      215.8869
f_all_Hz         340.5087      340.5087      340.5087


## Lattice potentials
The confinement from lattice beams is approximated by an spherical 3D-Gaussian.

In [96]:
# trap frequency calculations
f_x_1064lat = np.sqrt(2 * V_1064 / E9c.m_K40 / E9c.w0_lw**2) / 2 / np.pi
f_xy_1064lat = f_x_1064lat * np.sqrt(2)
# note that the 532 potential should have a minus sign when calculating rms
f_x_532lat = np.sqrt(2 * V_532 / E9c.m_K40 / E9c.w0_lw**2) / 2 / np.pi
f_xy_532lat = f_x_532lat * np.sqrt(2)

print(f"f_xy_1064lat = {f_xy_1064lat:.3f} Hz")
print(f"f_xy_532lat = {f_xy_532lat:.3f} Hz")

f_xy_1064lat = 72.600 Hz
f_xy_532lat = 0.000 Hz


## Hole closing rate
Using the rms trap frequency in the lattice plane for now, although I feel like different axes should be considered separately

In [99]:
str_for_trap = "f_xy_all_Hz"
row_idx = 2

In [100]:
f_trap_ODT = df[str_for_trap][row_idx]
f_trap = np.sqrt(f_trap_ODT**2 + f_xy_1064lat**2 - f_xy_532lat**2)
s_a_conv = df["axis_convention"][row_idx]
print(f"axis convention: {s_a_conv}")
print(f"{str_for_trap} = {f_trap_ODT:.2f} Hz")
print(f"including lattice confinement = {f_trap:.2f} Hz")
print(f"q_width = {q_width:.3f} k_BZ")

w_trap = 2 * np.pi * f_trap
for a_lat, lat_name in zip([E9c.a_lw_hex,       E9c.a_sw_tri],
                           ["1064 honeycomb",   "532 triangular"]):
    nu_trap = E9c.m_K40 * w_trap**2 * a_lat**2
    T_hole_recomb = E9c.hbar * np.pi**2 * q_width**2 / nu_trap
    T_ms_hole_recomb = T_hole_recomb * 1e3
    print(f"{lat_name}: {T_ms_hole_recomb:.4f} ms")

axis convention: x_ODTb_waist
f_xy_all_Hz = 215.89 Hz
including lattice confinement = 227.77 Hz
q_width = 0.200 k_BZ
1064 honeycomb: 0.6088 ms
532 triangular: 2.4352 ms
